#***Word2Vec***

In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# 질문과 두 개의 답변 정의
question = "항공우주산업의 재단 장학금을 받아야하는 이유는?"
answers = [
    "오늘 오후 비가 내리는 창밖을 바라보며, 커피 한 잔의 여유를 즐기는 오후. 사소한 일상의 순간들이 모여 하루를 완성시키는 것 같아요. 여기에 무언가를 더해 보자. 매일을 소중히 여기며 살아가는 것, 우리 삶에서 진정으로 의미 있는 일입니다.",
    "항공우주산업의 재단 장학금은 AI기술을 항공분야에 접목시켜 미래 신사업 발전을 지원하여, 제공되는 기회를 통해 나의 컴퓨터공학 전공을 확장하고 전문성을 향상시킬 수 있기 때문에 받아야 합니다."
]

# 단어 벡터 학습
all_sentences = [question] + answers
tokenized_sentences = [sentence.split() for sentence in all_sentences]
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# 문장 벡터 생성
def get_sentence_vector(sentence):
    word_vectors = [word2vec_model.wv[word] for word in sentence.split() if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# 질문과 답변 간의 유사도 계산
question_vector = get_sentence_vector(question)
answer_vectors = [get_sentence_vector(answer) for answer in answers]
similarity_scores = [cosine_similarity([question_vector], [answer_vector])[0,0] for answer_vector in answer_vectors]

print("질문과 첫 번째 답변 사이의 유사도:", similarity_scores[0])
print("질문과 두 번째 답변 사이의 유사도:", similarity_scores[1])


질문과 첫 번째 답변 사이의 유사도: 0.039609235
질문과 두 번째 답변 사이의 유사도: 0.18334134


#***TF-IDF***

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 질문과 두 개의 답변 정의
question = "항공우주산업의 재단 장학금을 받아야하는 이유는?"
answers = [
    "오늘 오후 비가 내리는 창밖을 바라보며, 커피 한 잔의 여유를 즐기는 오후. 사소한 일상의 순간들이 모여 하루를 완성시키는 것 같아요. 여기에 무언가를 더해 보자. 매일을 소중히 여기며 살아가는 것, 우리 삶에서 진정으로 의미 있는 일입니다. ",
    "항공우주산업의 재단 장학금은 AI기술을 항공분야에 접목시켜 미래 신사업 발전을 지원하여, 제공되는 기회를 통해 나의 컴퓨터공학 전공을 확장하고 전문성을 향상시킬 수 있기 때문에 받아야 합니다."
]

# 문서를 TF-IDF로 벡터화하여 코사인 유사도 계산
def get_similarity(question, docs):
    vectorizer = TfidfVectorizer().fit([question] + docs)
    vectors = vectorizer.transform([question] + docs)
    similarity_matrix = cosine_similarity(vectors)
    return similarity_matrix[0, 1:]

# 질문과 답변 간의 유사도 측정
similarity_scores = get_similarity(question, answers)

print("질문과 첫 번째 답변 사이의 유사도:", similarity_scores[0])
print("질문과 두 번째 답변 사이의 유사도:", similarity_scores[1])


질문과 첫 번째 답변 사이의 유사도: 0.0
질문과 두 번째 답변 사이의 유사도: 0.12053824250121294


#***CountVectorizer 임베딩***

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 질문과 두 개의 답변 정의
question = "항공우주산업의 재단 장학금을 받아야하는 이유는?"
answers = [
    "오늘 오후 비가 내리는 창밖을 바라보며, 커피 한 잔의 여유를 즐기는 오후. 사소한 일상의 순간들이 모여 하루를 완성시키는 것 같아요. 여기에 무언가를 더해 보자. 매일을 소중히 여기며 살아가는 것, 우리 삶에서 진정으로 의미 있는 일입니다. ",
    "항공우주산업의 재단 장학금은 AI기술을 항공분야에 접목시켜 미래 신사업 발전을 지원하여, 제공되는 기회를 통해 나의 컴퓨터공학 전공을 확장하고 전문성을 향상시킬 수 있기 때문에 받아야 합니다."]
# 문서를 벡터화하여 코사인 유사도 계산
def get_similarity(question, docs):
    vectorizer = CountVectorizer().fit_transform([question] + docs)
    vectors = vectorizer.toarray()
    similarity_matrix = cosine_similarity(vectors)
    return similarity_matrix[0, 1:]

# 질문과 답변 간의 유사도 측정
similarity_scores = get_similarity(question, answers)

print("질문과 첫 번째 답변 사이의 유사도:", similarity_scores[0])
print("질문과 두 번째 답변 사이의 유사도:", similarity_scores[1])


질문과 첫 번째 답변 사이의 유사도: 0.0
질문과 두 번째 답변 사이의 유사도: 0.18650096164806276


#***BERT***

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# BERT 모델 및 토크나이저 로드
model_name = 'bert-base-multilingual-cased'  # 다른 BERT 모델로 변경 가능
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 질문과 두 개의 답변 정의
question = "항공우주산업의 재단 장학금을 받아야하는 이유는?"
answers = [
    "오늘 오후 비가 내리는 창밖을 바라보며, 커피 한 잔의 여유를 즐기는 오후. 사소한 일상의 순간들이 모여 하루를 완성시키는 것 같아요. 여기에 무언가를 더해 보자. 매일을 소중히 여기며 살아가는 것, 우리 삶에서 진정으로 의미 있는 일입니다.",
    "항공우주산업의 재단 장학금은 AI기술을 항공분야에 접목시켜 미래 신사업 발전을 지원하여, 제공되는 기회를 통해 나의 컴퓨터공학 전공을 확장하고 전문성을 향상시킬 수 있기 때문에 받아야 합니다."
]

# 문장을 BERT 임베딩으로 변환하는 함수
def embed_sentence(sentence):
    # 문장을 토큰화하고 BERT 입력 형식에 맞게 변환
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    # BERT 모델에 입력 전달하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] 토큰의 임베딩을 문장 임베딩으로 반환
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# 질문과 답변을 BERT 임베딩으로 변환
question_embedding = embed_sentence(question)
answer_embeddings = [embed_sentence(answer) for answer in answers]

# 코사인 유사도 계산
def cosine_similarity(x, y):
    dot_product = torch.dot(torch.tensor(x).float(), torch.tensor(y).float())
    norm_x = torch.norm(torch.tensor(x).float())
    norm_y = torch.norm(torch.tensor(y).float())
    similarity = dot_product / (norm_x * norm_y)
    return similarity.item()

# 질문과 답변 간의 유사도 계산
similarity_scores = [cosine_similarity(question_embedding, answer_embedding) for answer_embedding in answer_embeddings]

print("질문과 첫 번째 답변 사이의 유사도:", similarity_scores[0])
print("질문과 두 번째 답변 사이의 유사도:", similarity_scores[1])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

질문과 첫 번째 답변 사이의 유사도: 0.6474209427833557
질문과 두 번째 답변 사이의 유사도: 0.7420502305030823
